In [1]:
import os
os.environ["XLA_PYTHON_CLIENT_PREALLOCATE"] = "false"
from tracr.rasp import rasp
from tracr.compiler import basis_inference
from tracr.compiler import craft_graph_to_model
from tracr.compiler import expr_to_craft_graph
from tracr.compiler import rasp_to_graph
from tracr.craft import bases
from tracr.rasp import rasp

from rasp_generator import utils, sampling
from tokenizer import tokenizer



In [2]:
sampler = sampling.ProgramSampler()
errs = sampler.sample()

In [3]:
graph, layers_to_nodes = tokenizer.rasp_to_layerwise_representation(sampler.program)

In [4]:
for layer, node_ids in layers_to_nodes.items():
    print(f"Layer {layer}")
    for node_id in node_ids:
        expr = graph.nodes[node_id]["EXPR"]
        if isinstance(expr, (rasp.Aggregate, rasp.SelectorWidth)):
            utils.print_expr(expr.selector)
        utils.print_expr(expr)
    print()

Layer 0

Layer 1
map_2 = Map(lambda x: x**0.5, tokens)    # type: float
sequence_map_5 = SequenceMap(lambda x, y: x*y, tokens, indices)    # type: categorical

Layer 2

Layer 3
sequence_map_4 = SequenceMap(lambda x, y: x*y, tokens, sequence_map_5)    # type: categorical

Layer 4

Layer 5
map_3 = Map(lambda x: x*2, sequence_map_4)    # type: float

Layer 6

Layer 7
linear_sequence_map_1 = LinearSequenceMap(map_2, map_3, 0.0, 0.0)    # type: float



In [5]:
utils.print_program(sampler.program)

map_2 = Map(lambda x: x**0.5, tokens)    # type: float
sequence_map_5 = SequenceMap(lambda x, y: x*y, tokens, indices)    # type: categorical
sequence_map_4 = SequenceMap(lambda x, y: x*y, tokens, sequence_map_5)    # type: categorical
map_3 = Map(lambda x: x*2, sequence_map_4)    # type: float
linear_sequence_map_1 = LinearSequenceMap(map_2, map_3, 0.0, 0.0)    # type: float
